# Distracted Driver Detection

## Goal

Classify driver behaviors using supervised and deep learning methods in order to prevent distracted driving incidents

## Dataset

[State Farm Distracted Driver Detection Dataset](https://www.kaggle.com/competitions/state-farm-distracted-driver-detection/data) isused for Distracted Driver Detection. The dataset consists of labelled photos of drivers taken inside the car.Dataset has ten different types of driver behaviours, including safe driving and various forms of distracted driving. These graphics depict ten different types of driver behaviours, including safe driving and various forms of distracted driving, such as texting, chatting on the phone, eating, and reaching behind.





### Import Packages

In [1]:
import os
import time
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns 
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.image as mpimg
from glob import glob
from tqdm import tqdm
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split  
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model,Sequential
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [2]:

os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
dataset = pd.read_csv('driver_imgs_list.csv')
dataset.head(5)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


### Preprocessing Data

In [4]:
NUMBER_CLASSES = 10
# Color type: 1 - grey, 3 - rgb

def get_cv2_image(path, img_rows, img_cols, color_type=3):
    # Loading as Grayscale image
    if color_type == 1:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    elif color_type == 3:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
    # Reduce size
    img = cv2.resize(img, (img_rows, img_cols)) 
    return img

# Training
def load_train(img_rows, img_cols, color_type=3):
    start_time = time.time()
    train_images = [] 
    train_labels = []
    # Loop over the training folder 
    for classed in tqdm(range(NUMBER_CLASSES)):
        print('Loading directory c{}'.format(classed))
        files = glob(os.path.join( 'imgs','train', 'c' + str(classed), '*.jpg'))
        print("*******files:",files)
        for file in files:
            print("-------",file)
            img = get_cv2_image(file, img_rows, img_cols, color_type)
            train_images.append(img)
            train_labels.append(classed)
    print("Data Loaded in {} second".format(time.time() - start_time))
    return train_images, train_labels 

def read_and_normalize_train_data(img_rows, img_cols, color_type):
    X, labels = load_train(img_rows, img_cols, color_type)
    y = to_categorical(labels, 10)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    x_train = np.array(x_train, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    x_test = np.array(x_test, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    
    return x_train, x_test, y_train, y_test

# Validation
def load_test(size=200000, img_rows=64, img_cols=64, color_type=3):
    path = os.path.join( 'imgs','test', '*.jpg')
    files = sorted(glob(path))
    X_test, X_test_id = [], []
    total = 0
    files_size = len(files)
    for file in tqdm(files):
        if total >= size or total >= files_size:
            break
        file_base = os.path.basename(file)
        img = get_cv2_image(file, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(file_base)
        total += 1
    return X_test, X_test_id

def read_and_normalize_sampled_test_data(size, img_rows, img_cols, color_type=3):
    test_data, test_ids = load_test(size, img_rows, img_cols, color_type)
    
    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(-1,img_rows,img_cols,color_type)
    
    return test_data, test_ids

In [5]:
img_rows = 64
img_cols = 64
color_type = 1

In [ ]:
#train-test split
x_train, x_test, y_train, y_test = read_and_normalize_train_data(img_rows, img_cols, color_type)
print('Train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

In [7]:
nb_test_samples = 200
test_files, test_targets = read_and_normalize_sampled_test_data(nb_test_samples, img_rows, img_cols, color_type)
print('Test shape:', test_files.shape)
print(test_files.shape[0], 'Test samples')

  0%|          | 200/79726 [00:02<18:24, 72.01it/s]

Test shape: (200, 64, 64, 1)
200 Test samples


### Data Visualization

In [8]:
# Statistics
# Load the list of names
names = [item[17:19] for item in sorted(glob("imgs/train/*/"))]
test_files_size = len(np.array(glob(os.path.join( 'imgs', 'test', '*.jpg'))))
x_train_size = len(x_train)
categories_size = len(names)
x_test_size = len(x_test)
print('There are %s total images.\n' % (test_files_size + x_train_size + x_test_size))
print('There are %d training images.' % x_train_size)
print('There are %d total training categories.' % categories_size)
print('There are %d validation images.' % x_test_size)
print('There are %d test images.'% test_files_size)

There are 102150 total images.

There are 17939 training images.
There are 10 total training categories.
There are 4485 validation images.
There are 79726 test images.


In [10]:
# Find the frequency of images per driver
drivers_id = pd.DataFrame((dataset['subject'].value_counts()).reset_index())
drivers_id.columns = ['driver_id', 'Counts']
drivers_id

,driver_id,Counts
0,p021,1237
1,p022,1233
2,p024,1226
3,p026,1196
4,p016,1078
5,p066,1034
6,p049,1011
7,p051,920
8,p014,876
9,p015,875


In [12]:
activity_map = {'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}

In [14]:
# Data Loading Function
def load_data(img_rows, img_cols):
    X, labels = load_train(img_rows, img_cols, color_type)
    y = to_categorical(labels, NUMBER_CLASSES)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    x_train = np.array(x_train, dtype=np.uint8).reshape(-1, img_rows, img_cols, color_type)
    x_test = np.array(x_test, dtype=np.uint8).reshape(-1, img_rows, img_cols, color_type)
    
    return x_train, x_test, y_train, y_test

In [15]:
# Load Data
x_train, x_test, y_train, y_test = load_data(img_rows, img_cols)

  0%|          | 0/10 [00:00<?, ?it/s]

Loading directory c0
*******files: ['imgs\\train\\c0\\img_100026.jpg', 'imgs\\train\\c0\\img_10003.jpg', 'imgs\\train\\c0\\img_100050.jpg', 'imgs\\train\\c0\\img_100074.jpg', 'imgs\\train\\c0\\img_10012.jpg', 'imgs\\train\\c0\\img_100145.jpg', 'imgs\\train\\c0\\img_100191.jpg', 'imgs\\train\\c0\\img_100257.jpg', 'imgs\\train\\c0\\img_100312.jpg', 'imgs\\train\\c0\\img_100337.jpg', 'imgs\\train\\c0\\img_100456.jpg', 'imgs\\train\\c0\\img_1005.jpg', 'imgs\\train\\c0\\img_10053.jpg', 'imgs\\train\\c0\\img_100542.jpg', 'imgs\\train\\c0\\img_100598.jpg', 'imgs\\train\\c0\\img_100605.jpg', 'imgs\\train\\c0\\img_100656.jpg', 'imgs\\train\\c0\\img_100665.jpg', 'imgs\\train\\c0\\img_100796.jpg', 'imgs\\train\\c0\\img_100824.jpg', 'imgs\\train\\c0\\img_100828.jpg', 'imgs\\train\\c0\\img_10092.jpg', 'imgs\\train\\c0\\img_100922.jpg', 'imgs\\train\\c0\\img_101015.jpg', 'imgs\\train\\c0\\img_101032.jpg', 'imgs\\train\\c0\\img_101091.jpg', 'imgs\\train\\c0\\img_101113.jpg', 'imgs\\train\\c0\\img_101

 10%|█         | 1/10 [00:03<00:29,  3.24s/it]

------- imgs\train\c0\img_94863.jpg
------- imgs\train\c0\img_94879.jpg
------- imgs\train\c0\img_94962.jpg
------- imgs\train\c0\img_94969.jpg
------- imgs\train\c0\img_95047.jpg
------- imgs\train\c0\img_95056.jpg
------- imgs\train\c0\img_95086.jpg
------- imgs\train\c0\img_95169.jpg
------- imgs\train\c0\img_9520.jpg
------- imgs\train\c0\img_95245.jpg
------- imgs\train\c0\img_95252.jpg
------- imgs\train\c0\img_95285.jpg
------- imgs\train\c0\img_95306.jpg
------- imgs\train\c0\img_9532.jpg
------- imgs\train\c0\img_95339.jpg
------- imgs\train\c0\img_95355.jpg
------- imgs\train\c0\img_95525.jpg
------- imgs\train\c0\img_95572.jpg
------- imgs\train\c0\img_95596.jpg
------- imgs\train\c0\img_95630.jpg
------- imgs\train\c0\img_95649.jpg
------- imgs\train\c0\img_95667.jpg
------- imgs\train\c0\img_95698.jpg
------- imgs\train\c0\img_957.jpg
------- imgs\train\c0\img_95728.jpg
------- imgs\train\c0\img_95735.jpg
------- imgs\train\c0\img_95760.jpg
------- imgs\train\c0\img_95774.

 20%|██        | 2/10 [00:06<00:24,  3.02s/it]

...

------- imgs\train\c7\img_96775.jpg
------- imgs\train\c7\img_96779.jpg
------- imgs\train\c7\img_96845.jpg
------- imgs\train\c7\img_96891.jpg
------- imgs\train\c7\img_96912.jpg
------- imgs\train\c7\img_96939.jpg
------- imgs\train\c7\img_96957.jpg
------- imgs\train\c7\img_96999.jpg
------- imgs\train\c7\img_97022.jpg
------- imgs\train\c7\img_97242.jpg
------- imgs\train\c7\img_9763.jpg
------- imgs\train\c7\img_97631.jpg
------- imgs\train\c7\img_9768.jpg
------- imgs\train\c7\img_97794.jpg
------- imgs\train\c7\img_97803.jpg
------- imgs\train\c7\img_97804.jpg
------- imgs\train\c7\img_97818.jpg
------- imgs\train\c7\img_9782.jpg
------- imgs\train\c7\img_97852.jpg
------- imgs\train\c7\img_9787.jpg
------- imgs\train\c7\img_97874.jpg
------- imgs\train\c7\img_97950.jpg
------- imgs\train\c7\img_98101.jpg
------- imgs\train\c7\img_9827.jpg
------- imgs\train\c7\img_98312.jpg
------- imgs\train\c7\img_98315.jpg
------- imgs\train\c7\img_98350.jpg
------- imgs\train\c7\img_98455.j

 90%|█████████ | 9/10 [00:26<00:02,  2.84s/it]

------- imgs\train\c8\img_94016.jpg
------- imgs\train\c8\img_94081.jpg
------- imgs\train\c8\img_94215.jpg
------- imgs\train\c8\img_94297.jpg
------- imgs\train\c8\img_94309.jpg
------- imgs\train\c8\img_94317.jpg
------- imgs\train\c8\img_94386.jpg
------- imgs\train\c8\img_94413.jpg
------- imgs\train\c8\img_94442.jpg
------- imgs\train\c8\img_94489.jpg
------- imgs\train\c8\img_9449.jpg
------- imgs\train\c8\img_94503.jpg
------- imgs\train\c8\img_94544.jpg
------- imgs\train\c8\img_94576.jpg
------- imgs\train\c8\img_9459.jpg
------- imgs\train\c8\img_94630.jpg
------- imgs\train\c8\img_9464.jpg
------- imgs\train\c8\img_94652.jpg
------- imgs\train\c8\img_9471.jpg
------- imgs\train\c8\img_94715.jpg
------- imgs\train\c8\img_94897.jpg
------- imgs\train\c8\img_94975.jpg
------- imgs\train\c8\img_9499.jpg
------- imgs\train\c8\img_95038.jpg
------- imgs\train\c8\img_95140.jpg
------- imgs\train\c8\img_95148.jpg
------- imgs\train\c8\img_95193.jpg
------- imgs\train\c8\img_95215.j

100%|██████████| 10/10 [00:29<00:00,  2.90s/it]

------- imgs\train\c9\img_99740.jpg
------- imgs\train\c9\img_99761.jpg
------- imgs\train\c9\img_99801.jpg
------- imgs\train\c9\img_99927.jpg
------- imgs\train\c9\img_9993.jpg
------- imgs\train\c9\img_99949.jpg
Data Loaded in 29.006038427352905 second


In [16]:
# Function to evaluate the model and return metrics
def evaluate_model_metrics(model, X_test, y_test):
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return accuracy, precision, recall, f1

# # Load models, evaluate metrics, and collect results
def load_and_evaluate_models(model_paths, img_rows, img_cols):
    metrics_table = []

    for fold_no, model_path in enumerate(model_paths, start=1):
        print(f"Evaluating Fold {fold_no}...")

        # Load the model
        model = load_model(model_path)

        # Evaluate on test data
        accuracy, precision, recall, f1 = evaluate_model_metrics(model, x_test, y_test)
        metrics_table.append([fold_no, accuracy, precision, recall, f1])

    return metrics_table

### Build CNN Model

In [17]:
# Define constants
NUMBER_CLASSES = 10
img_rows, img_cols, color_type = 64, 64, 3  
batch_size = 32
nb_epoch = 30
learning_rate = 0.0001
momentum = 0.9


def create_model():
    model = Sequential()

    # First convolution block
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(img_rows, img_cols, color_type), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second convolution block
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third convolution block
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Fourth convolution block
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model



In [18]:
# Data Loading Function
def load_data(img_rows, img_cols):
    X, labels = load_train(img_rows, img_cols, color_type)
    y = to_categorical(labels, NUMBER_CLASSES)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    x_train = np.array(x_train, dtype=np.uint8).reshape(-1, img_rows, img_cols, color_type)
    x_test = np.array(x_test, dtype=np.uint8).reshape(-1, img_rows, img_cols, color_type)
    
    return x_train, x_test, y_train, y_test

### Evaluation Functions

In [19]:
# Evaluation Function
def evaluate_model(model, X_test, y_test):
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=activity_map.values(), yticklabels=activity_map.values())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Function to plot training history
def plot_training_history(history, fold_no):
    
    # Accuracy plot
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Training and Validation Accuracy (Fold {fold_no})')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.show()

    # Loss plot
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss (Fold {fold_no})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

#  Table Display of Results
def print_table(metrics_table):
    table = PrettyTable()
    table.field_names = ["Fold", "Accuracy", "Precision", "Recall", "F1 Score"]

    for row in metrics_table:
        table.add_row([row[0], f"{row[1]:.4f}", f"{row[2]:.4f}", f"{row[3]:.4f}", f"{row[4]:.4f}"])

    print("\nEvaluation Metrics Table for Each Fold:")
    print(table)


### K-Fold Cross Validation

In [20]:
# Model Training with Cross-validation
def train_with_kfold(x_train, y_train, x_test, y_test, k=5):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_no = 1
    best_model = None
    best_accuracy = 0

    for train_idx, val_idx in kfold.split(x_train, y_train):
        print(f"Training Fold {fold_no}/{k}...")
        
        x_train_fold, x_val_fold = x_train[train_idx], x_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        model = create_model()
        
        # Callbacks
        checkpointer = ModelCheckpoint(f'saved_models/model_fold_{fold_no}.keras', save_best_only=True, verbose=1)
        es = EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=1)
        callbacks = [checkpointer, es]
        
        # Train the model
        history = model.fit(x_train_fold, y_train_fold, epochs=nb_epoch, batch_size=batch_size, validation_data=(x_val_fold, y_val_fold), callbacks=callbacks, verbose=1)
        
        # Evaluate on validation data
        val_loss, val_accuracy = model.evaluate(x_val_fold, y_val_fold, verbose=0)
        print(f"Validation Accuracy for Fold {fold_no}: {val_accuracy:.2f}")
        
        # Save the best model
        if val_accuracy > best_accuracy:
            best_model = model
            best_accuracy = val_accuracy
        
        # Plot training history
        plot_training_history(history, fold_no)
        
        fold_no += 1

    print(f"Best Validation Accuracy: {best_accuracy:.2f}")
    return best_model



### Evaluation with Best Model

In [ ]:
# Load Data
x_train, x_test, y_train, y_test = load_data(img_rows, img_cols)   

# Train and Evaluate Model
best_model = train_with_kfold(x_train, y_train, x_test, y_test, k=5)

# Final Evaluation on Test Set
print("\nEvaluating Best Model on Test Set...")
evaluate_model(best_model, x_test, y_test)

# Define paths to your saved models
model_paths = [
    "saved_models/model_fold_1.keras",
    "saved_models/model_fold_2.keras",
    "saved_models/model_fold_3.keras",
    "saved_models/model_fold_4.keras",
    "saved_models/model_fold_5.keras"
]

# Define image dimensions 
img_rows, img_cols = 48, 48  

# Evaluate metrics
metrics_table = load_and_evaluate_models(model_paths, img_rows, img_cols)
# Display Fancy Table
print_table(metrics_table)
